In [1]:
import pandas as pd 
import numpy as np
import joblib
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_parquet(r'../data/processed_data/modelo_dataset.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22257 entries, 0 to 22256
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      22257 non-null  object
 1   predictor  22257 non-null  object
dtypes: object(2)
memory usage: 347.9+ KB


Preparar y optimizar datos textuales para el modelo de recomendación

In [3]:
# Crear el vectorizador TF-IDF
vectorizer = TfidfVectorizer(min_df=4, max_df=0.85, ngram_range=(1, 2), max_features=40000, dtype=np.float32)

# Transformar la columna 'overview' en una matriz TF-IDF
matriz = vectorizer.fit_transform(df['predictor'])

# Reductir la dimensionalidad con SVD.
svd = TruncatedSVD(n_components=400, random_state=42)
matriz_reducida = svd.fit_transform(matriz)

# Contar características y uso de memoria
num_features = len(vectorizer.vocabulary_)

# Monitorearuso de memoria

import psutil
import os

def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / (1024 * 1024)  # Convertir de bytes a MB

print(f"Uso de memoria: {get_memory_usage()} MB")
print(f"El vectorizador está utilizando {num_features} características.")
print(f"Dimensiones reducidas a {matriz_reducida.shape[1]} componentes.")


Uso de memoria: 558.24609375 MB
El vectorizador está utilizando 40000 características.
Dimensiones reducidas a 400 componentes.


A continuacion se implementa una función de recomendación que utiliza la similitud de coseno para encontrar películas similares basándose en la representación vectorial de las descripciones (en este caso, la columna predictor). También se incluye un mecanismo para verificar el uso de memoria

In [4]:
def recommendacion(title):
    # Encontrar el índice de la película
    title = title.lower()
    idx = df.index[df['title'] == title].tolist()
    if not idx:
        return "Película no encontrada"
    idx = idx[0]
    
    # Calcular la matriz de similitud del coseno
    sim_scores = cosine_similarity(matriz_reducida[idx].reshape(1, -1), matriz_reducida)
    
    # Obtener y ordenar los puntajes de similitud
    sim_scores = list(enumerate(sim_scores[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtener las películas más similares
    movie_indices = [i[0] for i in sim_scores[1:6]]  # 10 películas más similares
    recomendaciones = df['title'].iloc[movie_indices].tolist()
    return {"Recomendaciones": recomendaciones}

# Prueba la función de recomendación
print(recommendacion('toy story'))

# Monitorear el uso de memoria

import psutil
import os

def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / (1024 * 1024)  # Convertir de bytes a MB

print(f"Uso de memoria: {get_memory_usage()} MB")

{'Recomendaciones': ['toy story 2', 'hawaiian vacation', 'toy story 3', 'small fry', 'partysaurus rex']}
Uso de memoria: 555.4609375 MB


In [7]:
directory = '../data/processed_data/'
if not os.path.exists(directory):
    os.makedirs(directory)

# Guardar el modelo TF-IDF y la matriz reducida
joblib.dump(vectorizer, os.path.join(directory, 'vectorizer.pkl'))
joblib.dump(matriz_reducida, os.path.join(directory, 'matriz_reducida.pkl'))

['../data/processed_data/matriz_reducida.pkl']

In [8]:
# Rutas de los archivos
vectorizer_path = '../data/processed_data/vectorizer.pkl'
matriz_reducida_path = '../data/processed_data/matriz_reducida.pkl'

# Función para obtener el tamaño de un archivo
def get_file_size(file_path):
    try:
        size = os.path.getsize(file_path)  # Obtener el tamaño en bytes
        return size / (1024 * 1024)  # Convertir a MB
    except FileNotFoundError:
        return "El archivo no existe."

# Obtener y mostrar el tamaño de los archivos
vectorizer_size = get_file_size(vectorizer_path)
matriz_reducida_size = get_file_size(matriz_reducida_path)

print(f"Tamaño del vectorizer: {vectorizer_size:.2f} MB")
print(f"Tamaño de la matriz reducida: {matriz_reducida_size:.2f} MB")

Tamaño del vectorizer: 1.34 MB
Tamaño de la matriz reducida: 33.96 MB
